In [1]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np 
import matplotlib.pyplot as plt

# inline magic changes figsize, set it again here
import matplotlib
matplotlib.rcParams['figure.figsize'] = 10,8 
default_color_list = plt.rcParams['axes.prop_cycle'].by_key()['color'] # get default colors

from astropy.table import Table

# 0. File Processing
This notebook creates `parameters.table` from the filenames of a folder, which is used to load the appropriate files later. We discover which PS and PC files are available, from the noiseless spectra.

In general, this file **does not need to be run** as the file `parameters.table` will be already generated.

In [3]:
PS_directory = 'powerspectrum_noiseless/'
PC_directory = 'peakcounts_noiseless/'

PS_names, PC_names = [], []
for filename in os.listdir(PS_directory):
    if filename.endswith("_z05.npy"):  
        PS_names.append(filename)
for filename in os.listdir(PC_directory):
    if filename.endswith("_z05.npy"): 
        PC_names.append(filename)
print('Found', len(PS_names), 'PS files and', len(PC_names), 'PC files.')

Found 101 PS files and 101 PC files.


## Table Construction
We now read the parameters in `cosmological_parameters.txt` and look for a file in the noiseless spectra corresponding to it, and similarly for peak counts. The $\Omega_m$ values between files and parameters do not match, due to different accounting of the neutrino contribution. The easiest way for me to do this was to filter by `As`, and handle the two exceptions (the two fiducial models) separately.

In [4]:
# helper function for extracting the numerical value of As from a filename
def get_As( string_with_As ):
    return float(string_with_As.split('_As')[1].split('_mva')[0])

# read in list of simulation cosmo parameters
table = Table.read('cosmological_parameters.txt', format='ascii')

# begin extracting filenames
PS_filename_list_temp = []
PC_filename_list_temp = []
for row in table:
    # special cases (fiducial 1 and 2)
    if '1a(fiducial)' in row['Model']:
        PS_name = 'Om0.29997_As2.10000_mva0.00000_mvb0.00000_mvc0.00000_h0.70000_Ode0.69995_PS_50_z05.npy'
        PC_name = 'Om0.29997_As2.10000_mva0.00000_mvb0.00000_mvc0.00000_h0.70000_Ode0.69995_PC_S_z05.npy'
    elif '1b(fiducial)' in row['Model']:
        PS_name = 'Om0.29780_As2.10000_mva0.02175_mvb0.02338_mvc0.05486_h0.70000_Ode0.69995_PS_50_z05.npy'
        PC_name = 'Om0.29780_As2.10000_mva0.02175_mvb0.02338_mvc0.05486_h0.70000_Ode0.69995_PC_S_z05.npy'
    else:
        PS_filename_candidates = [x for x in PS_names if np.isclose(row['10^9*A_s'],get_As(x), atol=1e-4)]
        PC_filename_candidates = [x for x in PC_names if np.isclose(row['10^9*A_s'],get_As(x), atol=1e-4)]
        if len(PS_filename_candidates) == 1 and len(PC_filename_candidates) == 1:
             PS_name, PC_name = PS_filename_candidates[0], PC_filename_candidates[0]
        else:
            raise ValueError('multiple file candidates found! possibly fiducial model 1a/1b')

    PS_filename_list_temp.append(PS_name)
    PC_filename_list_temp.append(PC_name)

# put the filenames in the table for easy access
table['PS'] = np.array(PS_filename_list_temp)
table['PC'] = np.array(PC_filename_list_temp)

In [5]:
table

Model,M_nu(eV),Omega_m,10^9*A_s,sigma_8(derived),PS,PC
str12,float64,float64,float64,float64,str86,str85
1a(fiducial),0.0,0.3,2.1,0.8523,Om0.29997_As2.10000_mva0.00000_mvb0.00000_mvc0.00000_h0.70000_Ode0.69995_PS_50_z05.npy,Om0.29997_As2.10000_mva0.00000_mvb0.00000_mvc0.00000_h0.70000_Ode0.69995_PC_S_z05.npy
1b(fiducial),0.1,0.3,2.1,0.8295,Om0.29780_As2.10000_mva0.02175_mvb0.02338_mvc0.05486_h0.70000_Ode0.69995_PS_50_z05.npy,Om0.29780_As2.10000_mva0.02175_mvb0.02338_mvc0.05486_h0.70000_Ode0.69995_PC_S_z05.npy
2,0.06271,0.3815,2.2004,1.0135,Om0.38016_As2.20037_mva0.00312_mvb0.00913_mvc0.05046_h0.70000_Ode0.61841_PS_50_z05.npy,Om0.38016_As2.20037_mva0.00312_mvb0.00913_mvc0.05046_h0.70000_Ode0.61841_PC_S_z05.npy
3,0.06522,0.2821,1.8826,0.7563,Om0.28062_As1.88255_mva0.00479_mvb0.00983_mvc0.05059_h0.70000_Ode0.71790_PS_50_z05.npy,Om0.28062_As1.88255_mva0.00479_mvb0.00983_mvc0.05059_h0.70000_Ode0.71790_PC_S_z05.npy
4,0.06773,0.4159,1.6231,0.9171,Om0.41442_As1.62306_mva0.00631_mvb0.01066_mvc0.05076_h0.70000_Ode0.58404_PS_50_z05.npy,Om0.41442_As1.62306_mva0.00631_mvb0.01066_mvc0.05076_h0.70000_Ode0.58404_PC_S_z05.npy
5,0.07024,0.2023,2.3075,0.6231,Om0.20080_As2.30753_mva0.00773_mvb0.01155_mvc0.05096_h0.70000_Ode0.79760_PS_50_z05.npy,Om0.20080_As2.30753_mva0.00773_mvb0.01155_mvc0.05096_h0.70000_Ode0.79760_PC_S_z05.npy
6,0.07275,0.3283,2.2883,0.9324,Om0.32666_As2.28829_mva0.00908_mvb0.01249_mvc0.05118_h0.70000_Ode0.67169_PS_50_z05.npy,Om0.32666_As2.28829_mva0.00908_mvb0.01249_mvc0.05118_h0.70000_Ode0.67169_PC_S_z05.npy
7,0.07526,0.3355,1.5659,0.7828,Om0.33386_As1.56595_mva0.01037_mvb0.01347_mvc0.05142_h0.70000_Ode0.66444_PS_50_z05.npy,Om0.33386_As1.56595_mva0.01037_mvb0.01347_mvc0.05142_h0.70000_Ode0.66444_PC_S_z05.npy
8,0.07778,0.2597,2.4333,0.8008,Om0.25795_As2.43331_mva0.01163_mvb0.01446_mvc0.05169_h0.70000_Ode0.74029_PS_50_z05.npy,Om0.25795_As2.43331_mva0.01163_mvb0.01446_mvc0.05169_h0.70000_Ode0.74029_PC_S_z05.npy


In [6]:
table.write('parameters.table', format='ascii', overwrite=True)